In [62]:
import requests
import osmnx as ox
import json
import geopandas as gpd
from shapely.geometry import mapping
import os
import numpy as np
from pyproj import Transformer

In [ ]:
TRANSFORMER_DEG = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
APIKEY = "my key"
ZONE = "Paris, France"
ROUNDABOUTS = None

PHOTO_MAX_RONDPOINT = 21

RAYON = 500
RAYON_DISCRIMINANT = 50 # en metres zone autour d'une coordonnée sans autre coordonnée
NBR_POSS = RAYON // RAYON_DISCRIMINANT

DIRECTIONS = [(1.0,0.0), (0.5, 0.5), (0.0,1.0), (-0.5, 0.5), (-1.0, 0.0), (-0.5, -0.5), (0.0,-1.0), (0.5, -0.5)]

LAMBDA = 0.8
PROBA_CALC = np.arange(NBR_POSS)
PROBA_CALC = np.exp(-LAMBDA * PROBA_CALC)
PROBA_CALC = PROBA_CALC / PROBA_CALC.sum()


ROUNDABOUTS = ox.features_from_place(ZONE, tags={'junction': 'roundabout'})

ROUNDABOUTS = ROUNDABOUTS.to_crs(epsg=3857)

ROUNDABOUTS['center'] = ROUNDABOUTS.centroid

ROUNDABOUTS = ROUNDABOUTS.reset_index(drop=True)
ROUNDABOUTS['id'] = (ROUNDABOUTS.index).astype(int)

ROUNDABOUTS = ROUNDABOUTS[['id', 'center', 'geometry']]

np.float64(0.9999999999999999)

In [ ]:
def pointgen(centre, x, y):
    x = centre[0] + x
    y = centre[1] + y
    point = (x, y)
    return point

In [ ]:
def choose_nbr(proba):
    testproba = 0.0
    for i in range(NBR_POSS):
        testproba += PROBA_CALC[i]
        if proba <= testproba:
            return i
    return i

0.5508558274262423
0.7983713057416106
0.9095871792160337
0.9595596924376882
0.9820137900379085
0.9921030664528023
0.996636470572999
0.9986734603502516
0.9995887388567819
0.9999999999999999


9

In [ ]:
def random_point():
    div_rayonmax = RAYON / 2
    tranche = np.random.choice(DIRECTIONS)
    proba = np.random.random()


    if (distx > 0):
        


In [ ]:
coordonnees = np.zeros((len(ROUNDABOUTS), PHOTO_MAX_RONDPOINT, 2), float)

angle_distance_possible = []

for i in ROUNDABOUTS['id']:
    centre = ROUNDABOUTS['center'][i]
    coordonnees[i, 0, 0], coordonnees[i, 0, 1] = TRANSFORMER_DEG.transform((ROUNDABOUTS['center'][i].x), (ROUNDABOUTS['center'][i].y))

    



In [73]:
features = []
for i, (_, row) in enumerate(ROUNDABOUTS.iterrows()):
    slice_coords = coordonnees[i]
    poly_list = slice_coords.tolist()
    features.append({
        "type": "Feature",
        "id": int(row['id']),
        "properties": {
            "center": [TRANSFORMER_DEG.transform(row['center'].x, row['center'].y)]
        },
        "geometry": {
        "type": "Polygon",
        "coordinates": [poly_list]
        }
    })

geojson = {"type": "FeatureCollection", "features": features}

with open("roundabouts.json", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"Nombre de ronds-points trouvés : {len(ROUNDABOUTS)}")


Nombre de ronds-points trouvés : 7


In [35]:
def recup_image_google_street_view(lat, lon, heading=0, fov=90, pitch=0, size="600x400", api_key=APIKEY):
    return f"https://maps.googleapis.com/maps/api/streetview?size={size}&location={lat},{lon}&fov={fov}&heading={heading}&pitch={pitch}&key={api_key}"

In [36]:
def recup_from_coord(lat, long, folder_name, id):
    image_path = os.path.join(folder_name, f"streetview_{id}.jpg")
    url = recup_image_google_street_view(lat, long)
    response = requests.get(url)
    if response.status_code == 200:
        with open(image_path, "wb") as img_file:
            img_file.write(response.content)
    else:
        print(f"Erreur lors du téléchargement pour le rond-point {id}")

In [51]:
for elem in geojson["features"]:
    id = elem["id"]
    geometry = elem["geometry"]
    folder_name = f"roundabout_{id}"
    os.makedirs(folder_name, exist_ok=True)

    if geometry['type'] == 'Polygon':
        polygone_coords = geometry['coordinates'][0]
    else:
        polygone_coords = geometry['coordinates']
        
    for i, coord in enumerate(polygone_coords):
        lat = coord[1]
        long = coord[0]
        recup_from_coord(lat, long, folder_name, i)